# Importing required modules

In [1]:
## importing packages and setting connector to snowflake
import pandas as pd
from drg_connect import Snowflake
pd.options.display.float_format = '{:.0f}'.format
pd.set_option('display.max_columns', None)
pd.set_option('display.height', 500)
pd.set_option('display.max_rows', 500)

import warnings
warnings.filterwarnings('ignore')

# Snowflake connection parameter

In [2]:
##defining parameters of snowflake
snow = Snowflake(role = 'RWD_RESTRICTED', warehouse = 'RWD_WH_ANALYTICS' ,database='RWD_DB',schema ='RWD_RESTRICTED')
engine = snow.engine

%reload_ext sql_magic
%config SQL.output_result = True  #Enable output to std out
%config SQL.notify_result = False #disable browser notifications
%config SQL.conn_name = 'engine'  #Set the sql_magic connection engine

# CMS OP Provider Table

In [27]:
%%read_sql
Create or replace table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1 as
    select A.HCPCS_CD as Procedures,
           B.DESY_SORT_KEY as Patient_id,                          
           B.Claim_No as claim_number,  
           B.ORG_NPI_NUM as facility_npi,  
           B.AT_PHYSN_NPI as physician_npi,  
           B.CLM_THRU_DT as service_date
      from RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_REVENUE_CENTER   A
      join RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_BASE_CLAIM       B
             on A.CLAIM_NO = B.CLAIM_NO               
      where trim(upper(HCPCS_CD)) in ('66940','66920','66982','66983','66984','G9389',
                           'G9390','66840','66850','66852','65175','66986',
                           '13.71','13.41','13.42','13.43','13.6','13.69',
                           '1371','1341','1342','1343','136','1369');

Query started at 01:20:48 PM India Standard Time; Query executed in 0.55 m

status
0  Table EYEPOINT_CATARACT_FAC_PHY_CMS_OP1 succes...

In [30]:
%%read_sql
SELECT MIN(service_date), MAX(service_date) 
from PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1

Query started at 01:24:57 PM India Standard Time; Query executed in 0.05 m

MIN(SERVICE_DATE) MAX(SERVICE_DATE)
0        2017-01-02        2017-12-31

In [28]:
%%read_sql
    SELECT COUNT(DISTINCT PATIENT_ID) as pat_cnt, 
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt, 
           COUNT(DISTINCT FACILITY_NPI) as fac_cnt, 
           COUNT(DISTINCT PHYSICIAN_NPI) as phy_cnt
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1

Query started at 01:21:27 PM India Standard Time; Query executed in 0.08 m

pat_cnt  clm_cnt  fac_cnt  phy_cnt
0   359771   546361     2606     7963

In [8]:
%%read_sql
    SELECT procedures,
           COUNT(DISTINCT PATIENT_ID) as pat_cnt, 
           COUNT(DISTINCT CLAIM_NUMBER) as clm_cnt, 
           COUNT(DISTINCT FACILITY_NPI) as fac_cnt, 
           COUNT(DISTINCT PHYSICIAN_NPI) as phy_cnt
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1
     GROUP BY procedures
     ORDER BY clm_cnt desc

Query started at 07:32:37 PM India Standard Time; Query executed in 0.11 m

procedures  pat_cnt  clm_cnt  fac_cnt  phy_cnt
0      66984   330597   496713     2521     6994
1      66982    35443    42763     1866     4143
2      66986     2548     2609      642     1318
3      66850     1498     1549      699     1075
4      66852     1369     1383      393      806
5      66983      553      635      186      238
6      66840      471      475      311      398
7      66940      292      297      162      197
8      66920      203      204      143      182
9      65175       37       37       32       37

In [5]:
%%read_sql
ALTER TABLE PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1
        ADD DOB_DT  VARCHAR,
            GENDER_code  VARCHAR,
            PATIENT_STATE_code   VARCHAR;

Query started at 02:18:44 PM India Standard Time; Query executed in 0.05 m

status
0  Statement executed successfully.

In [6]:
%%read_sql
UPDATE  PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1 A
                SET A.DOB_DT = B.DOB_DT,
                    A.GENDER_code = B.GNDR_CD,
                    A.PATIENT_STATE_code = B.BENE_STATE_CD
       FROM RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_BASE_CLAIM   B
        WHERE A.claim_number = B.CLAIM_NO

Query started at 02:19:04 PM India Standard Time; Query executed in 0.15 m

number of rows updated  number of multi-joined rows updated
0                  550804                                    0

In [8]:
%%read_sql
ALTER TABLE PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1
DROP column "DATE_OF_BIRTH", "GENDER", "PATIENT_STATE";

Query started at 02:23:23 PM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

# CMS ASC

In [9]:
%%read_sql
CREATE TABLE PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1 as
      select HCPCS as Procedures,
             SUPPLIER_ID_NUMBER as facility_npi,
             ALLOWED_SERVICES,
             '2016' as service_year
        from rwd_db.rwd_restricted.CMS_ASC_LDS_2016_ASC_2016
       where trim(upper(HCPCS)) in ('66940','66920','66982','66983','66984','G9389',
                           'G9390','66840','66850','66852','65175','66986',
                           '13.71','13.41','13.42','13.43','13.6','13.69',
                           '1371','1341','1342','1343','136','1369');

Query started at 07:32:53 PM India Standard Time; Query executed in 0.07 m

status
0  Table EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1 succe...

In [10]:
%%read_sql
    SELECT COUNT(DISTINCT FACILITY_NPI) as fac_cnt
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1

Query started at 07:33:29 PM India Standard Time; Query executed in 0.05 m

fac_cnt
0     1926

In [11]:
%%read_sql
    SELECT service_year,COUNT(DISTINCT FACILITY_NPI) as fac_cnt
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1 
      group by service_year

Query started at 07:33:35 PM India Standard Time; Query executed in 0.08 m

service_year  fac_cnt
0         2016     1926

In [12]:
%%read_sql
    SELECT procedures,COUNT(DISTINCT FACILITY_NPI) as fac_cnt
      FROM PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1
      group by procedures
      order by fac_cnt desc

Query started at 07:33:45 PM India Standard Time; Query executed in 0.05 m

procedures  fac_cnt
0      66984     1906
1      66982     1751
2      66986     1033
3      66840      681
4      66850      620
5      66852      243
6      66940      101
7      66983       38
8      65175       34
9      66920       27

# Grant permission

In [9]:
%%read_sql
GRANT ALL on table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_OP1 to role RWD_RESERVED_1

Query started at 02:23:32 PM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

In [37]:
%%read_sql
GRANT ALL on table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.EYEPOINT_CATARACT_FAC_PHY_CMS_ASC1 to role RWD_RESERVED_1

Query started at 05:48:36 PM India Standard Time; Query executed in 0.05 m

status
0  Statement executed successfully.

# Excel Tab3

In [42]:
%%read_sql
create or replace table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.cms_facility_clamins as
      select A.DESY_SORT_KEY as Patient_id,                          
             A.Claim_No as claim_number,  
             A.ORG_NPI_NUM as facility_npi,  
             A.CLM_THRU_DT as service_date,
             B.HCPCS_CD as Procedures,
             C.ICD_DGNS_CD as diagnosis
        from RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_BASE_CLAIM A
             left join RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_REVENUE_CENTER B
                    on A.CLAIM_NO = B.CLAIM_NO
             left join RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_BASE_CLAIM_DX C
                    on A.CLAIM_NO = C.CLAIM_NO
       where ORG_NPI_NUM in (select distinct facility_npi from PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.unique_fac);

Query started at 11:30:03 PM India Standard Time

Exception during reset or similar
Traceback (most recent call last):
  File "C:\Users\arajput\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool.py", line 703, in _finalize_fairy
    fairy._reset(pool)
  File "C:\Users\arajput\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\pool.py", line 873, in _reset
    pool._dialect.do_rollback(self)
  File "C:\Users\arajput\AppData\Local\Continuum\anaconda3\lib\site-packages\sqlalchemy\engine\default.py", line 457, in do_rollback
    dbapi_connection.rollback()
  File "C:\Users\arajput\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\connection.py", line 384, in rollback
    self.cursor().execute("ROLLBACK")
  File "C:\Users\arajput\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 479, in execute
    _is_put_get=_is_put_get)
  File "C:\Users\arajput\AppData\Local\Continuum\anaconda3\lib\site-packages\snowflake\connector\cursor.py", line 368, in _execute

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
; Query executed in 4.16 m

status
0  Table CMS_FACILITY_CLAMINS successfully created.

In [43]:
%%read_sql
GRANT ALL on table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.cms_facility_clamins to role RWD_RESERVED_1

Query started at 11:34:13 PM India Standard Time; Query executed in 0.06 m

status
0  Statement executed successfully.

In [44]:
%%read_sql
create or replace table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.cms_physician_clamins as
      select A.DESY_SORT_KEY as Patient_id,                          
             A.Claim_No as claim_number,  
             A.AT_PHYSN_NPI as physician_npi,  
             A.CLM_THRU_DT as service_date,
             B.HCPCS_CD as Procedures,
             C.ICD_DGNS_CD as diagnosis
        from RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_BASE_CLAIM A
             left join RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_REVENUE_CENTER B
                    on A.CLAIM_NO = B.CLAIM_NO
             left join RWD_DB.RWD_RESTRICTED.CMS_HOSP_LDS_2017Q4_OP_BASE_CLAIM_DX C
                    on A.CLAIM_NO = C.CLAIM_NO               
        where AT_PHYSN_NPI in (select distinct cast(physician_npi as string) from PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.unique_physician); 

Query started at 11:51:58 PM India Standard Time; Query executed in 0.48 m

status
0  Table CMS_PHYSICIAN_CLAMINS successfully created.

In [45]:
%%read_sql
GRANT ALL on table PROJECT_ANALYTICS.EYEPOINT_PHYFACTAR.cms_physician_clamins to role RWD_RESERVED_1

Query started at 11:52:27 PM India Standard Time; Query executed in 0.09 m

status
0  Statement executed successfully.